In [1]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('tiger.jpg')

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the range of red color in HSV
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])

# Threshold the HSV image to get only red color
mask = cv2.inRange(hsv, lower_red, upper_red)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(image, image, mask=mask)

# Display the image
cv2.imshow('image', image)
cv2.imshow('mask', mask)
cv2.imshow('res', res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('tiger.jpg')

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the range of red color in HSV
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])

# Threshold the HSV image to get only red color
mask = cv2.inRange(hsv, lower_red, upper_red)

blur = cv2.GaussianBlur(mask,(5,5),0)

# Find contours in the mask
contours, _ = cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    # Calculate the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Draw the bounding box onto the original image
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Display the image with bounding boxes
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import cv2
import numpy as np
import tensorflow_hub as hub
model = hub.load('https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1')

def load_image(image_path) -> tuple:
    image = cv2.imread(image_path)
    return image, None

def model_processing(image):
    pass

def convert_to_hsv(tuple) -> tuple:
    image, _ = tuple
    return image, cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

def get_red_mask(tuple) -> tuple:
    image, hsv = tuple
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    return image, cv2.inRange(hsv, lower_red, upper_red)

def apply_blur(tuple) -> tuple:
    image, mask = tuple
    return image, cv2.GaussianBlur(mask,(5,5),0)

def find_contours(tuple) -> tuple:
    image, blur = tuple
    contours, _ = cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return image, contours

def draw_bounding_boxes(tuple):
    image, contours = tuple
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return image

# Create the pipeline
pipeline = Pipeline([
    ('load_image', FunctionTransformer(load_image, validate=False)),
    ('convert_to_hsv', FunctionTransformer(convert_to_hsv, validate=False)),
    ('get_red_mask', FunctionTransformer(get_red_mask, validate=False)),
    ('apply_blur', FunctionTransformer(apply_blur, validate=False)),
    ('find_contours', FunctionTransformer(find_contours, validate=False)),
    ('draw_bounding_boxes', FunctionTransformer(draw_bounding_boxes, validate=False)),
])

# Use the pipeline to process an image
processed_image = pipeline.transform('tiger.jpg')

# Display the processed image
cv2.imshow('image', processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

model = hub.load('https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1')

def load_image(image_path) -> tuple:
    image = cv2.imread(image_path)
    return image, None

def convert_to_hsv(tuple) -> tuple:
    image, _ = tuple
    return image, cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

def get_red_mask(tuple) -> tuple:
    image, hsv = tuple
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    return image, cv2.inRange(hsv, lower_red, upper_red)

def apply_blur(tuple) -> tuple:
    image, mask = tuple
    return image, cv2.GaussianBlur(mask,(5,5),0)

def find_contours(tuple) -> tuple:
    image, blur = tuple
    contours, _ = cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return image, contours

def model_processing(tuple) -> tuple:
    image, contours = tuple
    # Convert the image to RGB and add a batch dimension
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rgb_tensor = tf.expand_dims(rgb, axis=0)
    # Run the model
    outputs = model(rgb_tensor)
    # Prepare a list for the model's bounding boxes
    model_boxes = []
    for i in range(int(outputs['num_detections'])):
        ymin, xmin, ymax, xmax = outputs['detection_boxes'][0][i].numpy()
        xmin *= image.shape[1]
        xmax *= image.shape[1]
        ymin *= image.shape[0]
        ymax *= image.shape[0]
        model_boxes.append([int(xmin), int(ymin), int(xmax), int(ymax)])
    return image, contours, model_boxes

def draw_model_boxes(tuple) -> tuple:
    image, contours, model_boxes = tuple
    for box in model_boxes:
        cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)
    return image, contours

def draw_color_boxes(tuple):
    image, contours = tuple
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return image

# Create the pipeline
pipeline = Pipeline([
    ('load_image', FunctionTransformer(load_image, validate=False)),
    ('convert_to_hsv', FunctionTransformer(convert_to_hsv, validate=False)),
    ('get_red_mask', FunctionTransformer(get_red_mask, validate=False)),
    ('apply_blur', FunctionTransformer(apply_blur, validate=False)),
    ('find_contours', FunctionTransformer(find_contours, validate=False)),
    ('model_processing', FunctionTransformer(model_processing, validate=False)),
    ('draw_model_boxes', FunctionTransformer(draw_model_boxes, validate=False)),
    ('draw_color_boxes', FunctionTransformer(draw_color_boxes, validate=False)),
])

# Use the pipeline to process an image
processed_image = pipeline.transform('tiger.jpg')

# Display the processed image
cv2.imshow('image', processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


2023-08-30 00:28:56.497831: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_tensor' with dtype uint8 and shape [1,?,?,3]
	 [[{{node input_tensor}}]]


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

class ProcessedImage:
    def __init__(self, image_path:str):
        self.image_path = image_path
        self.model = hub.load('https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1')
        self.PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
        self.category_index = label_map_util.create_category_index_from_labelmap(self.PATH_TO_LABELS, use_display_name=True)

    def load_image(self) -> tuple:
        image = cv2.imread(self.image_path)
        return image, None

    def convert_to_hsv(self, tuple) -> tuple:
        image, _ = tuple
        return image, cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    def get_red_mask(self, tuple) -> tuple:
        image, hsv = tuple
        lower_red = np.array([0, 50, 50])
        upper_red = np.array([10, 255, 255])
        return image, cv2.inRange(hsv, lower_red, upper_red)

    def apply_blur(self, tuple) -> tuple:
        image, mask = tuple
        return image, cv2.GaussianBlur(mask,(5,5),0)

    def find_contours(self, tuple) -> tuple:
        image, blur = tuple
        contours, _ = cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return image, contours

    def model_processing(self, tuple):
        image, _ = tuple
        # Convert the image to RGB and add a batch dimension
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        rgb_tensor = tf.expand_dims(rgb, axis=0)
        # Run the model
        outputs = self.model(rgb_tensor)
        # Prepare a list for the model's bounding boxes, classes, and scores
        self.model_boxes = outputs['detection_boxes'][0].numpy()
        self.model_classes = (outputs['detection_classes'][0].numpy() + 1).astype(int)
        self.model_scores = outputs['detection_scores'][0].numpy()
        return image, (self.model_boxes, self.model_classes, self.model_scores)

    def draw_model_boxes(self, tuple):
        image, (model_boxes, model_classes, model_scores) = tuple
        # Convert the image to RGB for visualization
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Draw the labels on the image
        viz_utils.visualize_boxes_and_labels_on_image_array(
            image_rgb,
            model_boxes,
            model_classes,
            model_scores,
            self.category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.30,
            agnostic_mode=False,
        )
        # Convert the image back to BGR for further processing
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        return image_bgr, None

    def draw_color_boxes(self, tuple):
        image, contours = tuple
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        return image

    def main(self):
    # Create the pipeline
        pipeline = Pipeline([
            ('load_image', FunctionTransformer(load_image, validate=False)),
            ('convert_to_hsv', FunctionTransformer(convert_to_hsv, validate=False)),
            ('get_red_mask', FunctionTransformer(get_red_mask, validate=False)),
            ('apply_blur', FunctionTransformer(apply_blur, validate=False)),
            ('find_contours', FunctionTransformer(find_contours, validate=False)),
            ('model_processing', FunctionTransformer(model_processing, validate=False)),
            ('draw_model_boxes', FunctionTransformer(draw_model_boxes, validate=False)),
            ('draw_color_boxes', FunctionTransformer(draw_color_boxes, validate=False)),
        ])

        # Use the pipeline to process an image
        processed_image = pipeline.transform('tiger.jpg')

        # Display the processed image
        cv2.imshow('image', processed_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
if __name__ == "__main__":
    processed_image = ProcessedImage()
    processed_image.main()
